In [3]:
import xml.etree.ElementTree as ET
from datetime import datetime
from dateutil.relativedelta import relativedelta
from elasticsearch import Elasticsearch, helpers
from getpass import getpass

In [4]:
es = Elasticsearch(
    getpass("Host: "),
    api_key=getpass("API Key: "),
)

Host:  ········
API Key:  ········


In [ ]:
tree = ET.parse('sample_data.xml')
root = tree.getroot()
three_months_ago = datetime.now() - relativedelta(months=3)

In [6]:
docs = []

In [7]:
for workout in root.findall('Workout'):
    start_date_str = workout.attrib['startDate']
    start_date = datetime.strptime(start_date_str[:19], "%Y-%m-%d %H:%M:%S")

    if start_date >= three_months_ago:
        distance_miles = 0.0
        calories = 0.0

        for stat in workout.findall('WorkoutStatistics'):
            if stat.attrib['type'] == "HKQuantityTypeIdentifierDistanceWalkingRunning":
                distance_miles = float(stat.attrib.get('sum', 0))
            elif stat.attrib['type'] == "HKQuantityTypeIdentifierActiveEnergyBurned":
                calories = float(stat.attrib.get('sum', 0))

        doc = {
            'workout_type': workout.attrib.get('workoutActivityType', 'Unknown'),
            'start_time': workout.attrib['startDate'],
            'end_time': workout.attrib['endDate'],
            'distance_miles': distance_miles,
            'distance_km': distance_miles * 1.60934,
            'calories_burned': calories,
            'device': workout.attrib.get('sourceName', 'Unknown')
        }
        
        docs.append(doc)

In [8]:
if docs:
    actions = [
        {
            "_index": "apple-health-workouts",
            "_source": doc
        }
        for doc in docs
    ]
    helpers.bulk(es, actions)
    print(f"Successfully indexed {len(docs)} workouts into Elasticsearch!")
else:
    print("No workouts found in the last 3 months.")

Successfully indexed 25 workouts into Elasticsearch!
